In [2]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.double_sorting import calculate_period_return
from environ.process.asset_pricing.fama_macbeth import build_dominance_factor, calculate_weekly_returns

def reg_fama_macbeth(data_fama_macbeth, formula:str = "dollar_ret ~ DOM"
) -> pd.DataFrame:
  risk_premia = (data_fama_macbeth
    .groupby(["Year", "Week"])
    .apply(lambda x: sm.ols(
        formula= formula, 
        data=x
      ).fit()
      .params
    )
    .reset_index()
  )

  price_of_risk = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")["estimate"]
    .apply(lambda x: pd.Series({
        "risk_premium": x.mean(),
        "t_stat": x.mean()/x.std()*np.sqrt(len(x))
      })
    )
    .reset_index()
    .pivot(index="factor", columns="level_1", values="estimate")
    .reset_index()
  )

  price_of_risk_NW = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")
    .apply(lambda x: (
        x["estimate"].mean()/ 
          sm.ols("estimate ~ 1", x)
          .fit(cov_type="HAC", cov_kwds={"maxlags": 4}).bse
      )
    )
    .reset_index()
    .rename(columns={"Intercept": "t_stat_NW"})
  )
  # express risk premium in percentage
  price_of_risk['risk_premium'] = price_of_risk['risk_premium'] * 100
  return price_of_risk.merge(price_of_risk_NW, on="factor").round(3)

In [30]:
# reg_panel = pd.read_pickle(
#     PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
# )
# reg_panel.columns

In [7]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

reg_panel["daily_supply_return"] = reg_panel["supply_rates"] / 365.2425


def get_weekly_panel(reg_panel):
    ### Get test assets returns 
    reg_panel['Date'] = pd.to_datetime(reg_panel['Date'])
    reg_panel['Year'] = reg_panel['Date'].dt.year
    reg_panel['Week'] = reg_panel['Date'].dt.isocalendar().week

    # add daily simple returns and convenience yield
    reg_panel = calculate_period_return(df_panel=reg_panel, freq=1, simple_dollar_ret=False)

    # Change into weekly returns
    reg_panel = calculate_weekly_returns(df_panel=reg_panel)

    reg_panel['weekly_dollar_ret'] = reg_panel.groupby(['Year', 'Week'])['weekly_dollar_ret'].transform(
            lambda x: x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99))
    )
    return reg_panel
    
# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    # "all": reg_panel,
}

for panel_info, df_panel in stable_nonstable_info.items():
    for simple_dollar_ret in [True]:
        for dominance in DEPENDENT_VARIABLES + ['Inflow_centrality', 'Outflow_centrality', 'TVL_share']: 
            print(f"Fama McBeth {panel_info} {simple_dollar_ret} {dominance}")
            dominance_factor = build_dominance_factor(
                    reg_panel=df_panel,
                    brk_pt_lst=[0.8],
                    dominance_var=dominance,
                    zero_value_portfolio=False,
                    simple_dollar_ret=simple_dollar_ret,
                )
            dominance_factor.drop(['P1', 'P2'], axis=1, inplace=True)
            dominance_factor.rename(columns={"P2 - P1": "DOM"}, inplace=True)

            assets_panel = get_weekly_panel(df_panel)
            # merge all factors
            data_fama_macbeth = pd.merge(dominance_factor, ff3, on=['Year', 'Week'], how="left")
            data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=['Year', 'Week'], how="left")
            # merge factors with returns
            data_fama_macbeth = pd.merge(data_fama_macbeth, assets_panel, on=['Year', 'Week'], how="left")
            data_fama_macbeth.rename(columns={"weekly_dollar_ret": "dollar_ret", "weekly_ret":"convenience_ret"}, inplace=True)
            data_fama_macbeth = data_fama_macbeth.dropna()

            # run the Fama-Macbeth regression
            data_fama_macbeth['excess_ret'] = data_fama_macbeth['dollar_ret'] - data_fama_macbeth['RF']
            fama_macbeth = reg_fama_macbeth(data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + DOM")
            display(fama_macbeth)


Fama McBeth stablecoin True volume_ultimate_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,-0.291,-0.827,-0.864
1,CMOM,0.857,0.985,1.010
2,CSIZE,-0.388,-1.038,-1.057
3,DOM,-0.016,-1.284,-1.319
4,Intercept,10.599,0.938,0.963


Fama McBeth stablecoin True eigen_centrality_undirected


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,-0.291,-0.827,-0.864
1,CMOM,0.857,0.985,1.010
2,CSIZE,-0.388,-1.038,-1.057
3,DOM,-0.019,-1.517,-1.525
4,Intercept,10.599,0.938,0.963


Fama McBeth stablecoin True vol_inter_full_len_share


...

,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.824,5.722,5.666
1,CMOM,0.001,0.007,0.006
2,CSIZE,0.270,3.248,2.413
3,DOM,0.087,0.663,0.696
4,Intercept,3.702,3.200,2.342


Fama McBeth non-stablecoin True Outflow_centrality


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.824,5.723,5.666
1,CMOM,0.001,0.007,0.006
2,CSIZE,0.270,3.247,2.413
3,DOM,0.087,0.664,0.698
4,Intercept,3.702,3.200,2.342


Fama McBeth non-stablecoin True TVL_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.816,5.693,5.624
1,CMOM,-0.002,-0.026,-0.024
2,CSIZE,0.263,3.195,2.382
3,DOM,-0.103,-0.673,-0.701
4,Intercept,3.746,3.259,2.373
